In [2]:
import pandas as pd
import re
import numpy as np

import requests
import urllib.request
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 세무정보

In [174]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((219, 7), dtype = np.int), columns=['category', 'title','writer','date','text','hit','url'])

In [175]:
df.head(3)

,category,title,writer,date,text,hit,url
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0


In [176]:
url = "https://omoney.kbstar.com/quics?page=C042014#loading"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=22:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(15) # 15초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find_all("td", class_ = "left")
    for tag in anchor_tags:
        story = tag.find("a")['href']
        # 링크를 클릭하여 글 내용 페이지로 이동
        url_in = "https://omoney.kbstar.com/"+story
        driver.get(url_in)

        # 글 내용이 로드될 때까지 잠시 대기
        driver.implicitly_wait(15) # 10초간 대기

        # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
        content_html = driver.page_source
        content_soup = BeautifulSoup(content_html, 'html.parser')

        # 글 내용 추출
        category = content_soup.find("div", class_ = "toparea").find("h1").text
        title = content_soup.find("dl", class_ = "board_view").find("strong").text
        writer = content_soup.find("span", class_ = "day_hit").find_all("span", class_="date")[0].text[3:]
        views = int(content_soup.find("span", class_ = "day_hit").text.split("\n")[-2][3:])
        date = content_soup.find("span", class_ = "day_hit").find_all("span", class_="date")[1].text[3:]
        txt = content_soup.find("dd", class_ = "cont").text.replace("\n","").replace("\t","").split("[출처")[0].strip()
        
        df.iloc[i,0] = category
        df.iloc[i,1] = title
        df.iloc[i,2] = writer
        df.iloc[i,3] = date
        df.iloc[i,4] = txt
        df.iloc[i,5] = views
        df.iloc[i,6] = url_in
        i+=1
        # 이전 페이지로 돌아가기
        driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        driver.get(url)
        if (page_num <= 10):
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 11):
            driver.find_element(By.NAME, "nextinput").click()
        elif (page_num <= 20):
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 21):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
        else:
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
    except Exception as e:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 10
2 페이지를 가져왔습니다 i: 20
3 페이지를 가져왔습니다 i: 30
4 페이지를 가져왔습니다 i: 40
5 페이지를 가져왔습니다 i: 50
6 페이지를 가져왔습니다 i: 60
7 페이지를 가져왔습니다 i: 70
8 페이지를 가져왔습니다 i: 80
9 페이지를 가져왔습니다 i: 90
10 페이지를 가져왔습니다 i: 100
11 페이지를 가져왔습니다 i: 110
12 페이지를 가져왔습니다 i: 120
13 페이지를 가져왔습니다 i: 130
14 페이지를 가져왔습니다 i: 140
15 페이지를 가져왔습니다 i: 150
16 페이지를 가져왔습니다 i: 160
17 페이지를 가져왔습니다 i: 170
18 페이지를 가져왔습니다 i: 180
19 페이지를 가져왔습니다 i: 190
20 페이지를 가져왔습니다 i: 200
21 페이지를 가져왔습니다 i: 210
22 페이지를 가져왔습니다 i: 219
23 페이지를 찾을 수 없습니다.


In [177]:
df.head()

,category,title,writer,date,text,hit,url
0,세무정보,"은퇴 후 부담되는 건강보험료, 면제되는 조건은?",이호용,2023.07.20,오랜 회사생활을 마치고 은퇴를 앞둔 직장인들은 소득이 줄어든 상태에서 고정적으로 나...,74107,https://omoney.kbstar.com/quics?page=C042014&b...
1,세무정보,현재 1주택이 있고 추가로 주택을 구입하면 취득세가 많이 나올까요?,김윤정,2023.07.12,현재 1주택이 있고 추가로 주택을 구입한다면?혹시 1세대 2주택이 되어 취득세가 많...,101047,https://omoney.kbstar.com/quics?page=C042014&b...
2,세무정보,주택 1채를 무주택자인 자녀에게 3억정도 싸게 팔아도 된다던데 맞을까요?,김윤정,2023.07.07,1세대 2주택인데 이 중 1채를 무주택자인 자녀에게 넘기려고 합니다.① (저가매매)...,106590,https://omoney.kbstar.com/quics?page=C042014&b...
3,세무정보,"내 금융소득과 건강보험료, 따져 보셨나요?",김재 센터장,2023.07.05,오늘 주제 한눈에 보기 퇴직을 앞두고 가장 먼저 준비해야 하는 것은 재무적인 항목이...,68400,https://omoney.kbstar.com/quics?page=C042014&b...
4,세무정보,1세대 1주택이 되면 재산세나 종합부동산세가 줄어드나요?,김윤정,2023.07.04,Q. 보유 주택이 2채인데 이 중 1채를 매각해서 1세대 1주택이 되면 재산세나 종...,104297,https://omoney.kbstar.com/quics?page=C042014&b...


In [178]:
df.tail()

,category,title,writer,date,text,hit,url
214,세무정보,3년 보유 2년 거주인가? 1년 거주인가?,박영선,2004.09.22,PB사업팀 전담 세무사 / 박영선 ...,240,https://omoney.kbstar.com/quics?page=C042014&b...
215,세무정보,재산세 조세저항의 원인과 전망,이장건,2004.09.07,PB 사업팀 전담 세무사 이장건) ...,174,https://omoney.kbstar.com/quics?page=C042014&b...
216,세무정보,"임대 주택, 이럴 때는 과세된다",윤영주,2004.08.06,윤영주(전담세무사) / GOLD&WISE PB 사업팀 ...,378,https://omoney.kbstar.com/quics?page=C042014&b...
217,세무정보,상속이 임박했을 때 할 일과 하지 말아야 할 일,이장건,2004.05.11,이장건 / GOLD&WISE 전담 세무사 Do's ...,834,https://omoney.kbstar.com/quics?page=C042014&b...
218,세무정보,2004년 이후 부동산 양도?상속?증여시 유의해야 할 사항,이장건,2004.05.06,이장건(전담세무사) / GOLD&WISE PB사업팀 2...,573,https://omoney.kbstar.com/quics?page=C042014&b...


In [179]:
df.to_csv('KB자산관리_세무.csv', index = False, encoding = 'utf-8')

## 부동산정보

In [167]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((197, 7), dtype = np.int), columns=['category', 'title','writer','date','text','hit','url'])

In [168]:
df.head(3)

,category,title,writer,date,text,hit,url
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0


In [169]:
url = "https://omoney.kbstar.com/quics?page=C042015#loading"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=20:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(15) # 15초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find_all("td", class_ = "left")
    for tag in anchor_tags:
        story = tag.find("a")['href']
        # 링크를 클릭하여 글 내용 페이지로 이동
        url_in = "https://omoney.kbstar.com/"+story
        driver.get(url_in)

        # 글 내용이 로드될 때까지 잠시 대기
        driver.implicitly_wait(15) # 10초간 대기

        # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
        content_html = driver.page_source
        content_soup = BeautifulSoup(content_html, 'html.parser')

        # 글 내용 추출
        category = content_soup.find("div", class_ = "toparea").find("h1").text
        title = content_soup.find("dl", class_ = "board_view").find("strong").text
        writer = content_soup.find("span", class_ = "day_hit").find_all("span", class_="date")[0].text[3:]
        views = int(content_soup.find("span", class_ = "day_hit").text.split("\n")[-2][3:])
        date = content_soup.find("span", class_ = "day_hit").find_all("span", class_="date")[1].text[3:]
        txt = content_soup.find("dd", class_ = "cont").text.replace("\n","").replace("\t","").split("[출처")[0].strip()
            
        df.iloc[i,0] = category
        df.iloc[i,1] = title
        df.iloc[i,2] = writer
        df.iloc[i,3] = date
        df.iloc[i,4] = txt
        df.iloc[i,5] = views
        df.iloc[i,6] = url_in
        i+=1
        # 이전 페이지로 돌아가기
        driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        driver.get(url)
        if (page_num <= 10):
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 11):
            driver.find_element(By.NAME, "nextinput").click()
        else:
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
    except Exception as e:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 10
2 페이지를 가져왔습니다 i: 20
3 페이지를 가져왔습니다 i: 30
4 페이지를 가져왔습니다 i: 40
5 페이지를 가져왔습니다 i: 50
6 페이지를 가져왔습니다 i: 60
7 페이지를 가져왔습니다 i: 70
8 페이지를 가져왔습니다 i: 80
9 페이지를 가져왔습니다 i: 90
10 페이지를 가져왔습니다 i: 100
11 페이지를 가져왔습니다 i: 110
12 페이지를 가져왔습니다 i: 120
13 페이지를 가져왔습니다 i: 130
14 페이지를 가져왔습니다 i: 140
15 페이지를 가져왔습니다 i: 150
16 페이지를 가져왔습니다 i: 160
17 페이지를 가져왔습니다 i: 170
18 페이지를 가져왔습니다 i: 180
19 페이지를 가져왔습니다 i: 190
20 페이지를 가져왔습니다 i: 197
21 페이지를 찾을 수 없습니다.


In [170]:
df.head()

,category,title,writer,date,text,hit,url
0,부동산정보,보증금 깎아줄게요! 전월세 갱신계약 줄었지만 감액계약 비중 역대 최대,KB부동산,2023.07.26,서울 아파트 시세 하락이 안정을 찾고 있습니다.다만 강남권역 중심의 인기지역 외 하...,28106,https://omoney.kbstar.com/quics?page=C042015&b...
1,부동산정보,"아파트 시장의 반등, 부동산 회복될까?",박원갑,2023.07.19,"아파트 시장의 반등, 부동산 회복될까?그동안 꽁꽁 얼어붙었던 아파트 시장에 온기가 ...",50977,https://omoney.kbstar.com/quics?page=C042015&b...
2,부동산정보,박원갑 박사가 알려주는 부동산 뉴스와 광고를 제대로 읽는 법,박원갑,2023.07.06,☞ 유튜브 여의도5번출구 바로가기 ☜ '암 수술 성공 확률이 95%입니다.''암 ...,52743,https://omoney.kbstar.com/quics?page=C042015&b...
3,부동산정보,"서울 아파트값 평균 매매가격 11억대 진입, 부동산 시장의 현주소는?",골든라이프X 게시글,2023.06.23,오늘 주제 한눈에 보기서울 아파트 평균 매매가격이 11억대로 내려앉았습니다. 전국 ...,52788,https://omoney.kbstar.com/quics?page=C042015&b...
4,부동산정보,[청약제도 개편] 청약 가점이 낮아도 당첨 기회가 있다고요?,이창섭 부동산 전문위원,2023.06.20,Q1. 낮은 점수로 청약 당첨이 가능하다고요?청약제도 개편으로 가점이 낮은 세대의 ...,79587,https://omoney.kbstar.com/quics?page=C042015&b...


In [171]:
df.tail()

,category,title,writer,date,text,hit,url
192,부동산정보,불황기에는 경매투자가 제격이다,박합수,2004.10.08,GOLD&WISE 부동산 팀장 박합수) 부동산 경매시장 주목 경기침체로...,359,https://omoney.kbstar.com/quics?page=C042015&b...
193,부동산정보,외환 위기 시절에 배운 불황기 부동산 운영전략,김희선,2004.09.07,부동산 114 김희선전무이사) 나홀...,246,https://omoney.kbstar.com/quics?page=C042015&b...
194,부동산정보,하반기 부동산시장의 전망,박합수,2004.08.06,박합수 / GOLD&WISE PB 사업팀 ...,316,https://omoney.kbstar.com/quics?page=C042015&b...
195,부동산정보,새롭게 등장한 아파텔 투자의 허와 실,김희선,2004.05.11,부동산114 김희선전무이사 10.29조치이후 얼어붙...,542,https://omoney.kbstar.com/quics?page=C042015&b...
196,부동산정보,상가투자에 대하여,박합수,2004.05.06,박합수 / GOLD&WISE PB사업팀 최근 들어 아...,492,https://omoney.kbstar.com/quics?page=C042015&b...


In [173]:
df.to_csv('KB자산관리_부동산.csv', index = False, encoding = 'utf-8')

## 재테크정보

In [217]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((462, 7), dtype = np.int), columns=['category', 'title','writer','date','text','hit','url'])

In [218]:
df.head()

,category,title,writer,date,text,hit,url
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [201]:
url = "https://omoney.kbstar.com/quics?page=C042016#loading"
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [216]:
for j in range(10):
    print(soup.find_all("td")[j*4+2].text.replace("\n","").replace("\t","").strip())

곽재혁
노지원
이주영 센터장
오인석
진은경 센터장
조옥순
곽재혁
최철호
신동준
곽재혁


In [225]:
url = "https://omoney.kbstar.com/quics?page=C042016#loading"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=47:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(15) # 15초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    for j in range(10):
        if (page_num == 47 and j>=2):
            break
        df.iloc[i+j,2] = soup.find_all("td")[j*4+2].text.replace("\n","").replace("\t","").strip()
    
    anchor_tags = soup.find_all("td", class_ = "left")
    for tag in anchor_tags:
        story = tag.find("a")['href']
        # 링크를 클릭하여 글 내용 페이지로 이동
        url_in = "https://omoney.kbstar.com/"+story
        driver.get(url_in)

        # 글 내용이 로드될 때까지 잠시 대기
        driver.implicitly_wait(15) # 10초간 대기

        # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
        content_html = driver.page_source
        content_soup = BeautifulSoup(content_html, 'html.parser')

        # 글 내용 추출
        category = content_soup.find("div", class_ = "toparea").find("h1").text
        title = content_soup.find("dl", class_ = "board_view").find("strong").text
        views = int(content_soup.find("span", class_ = "day_hit").text.split("\n")[-2][3:])
        date = content_soup.find("span", class_ = "day_hit").find("span", class_="date").text[3:]
        txt = content_soup.find("dd", class_ = "cont").text.replace("\n","").replace("\t","").split("[출처")[0].strip()
        
        df.iloc[i,0] = category
        df.iloc[i,1] = title
        df.iloc[i,3] = date
        df.iloc[i,4] = txt
        df.iloc[i,5] = views
        df.iloc[i,6] = url_in
        i+=1
        # 이전 페이지로 돌아가기
        driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        driver.get(url)
        if (page_num <= 10):
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 11):
            driver.find_element(By.NAME, "nextinput").click()
        elif (page_num <= 20):
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 21):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
        elif (page_num <=30):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 31):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
        elif (page_num <= 40):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
        elif (page_num == 41):
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
        else:
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            driver.find_element(By.NAME, "nextinput").click()
            page_name = 'pageinput'+str(page_num)
            driver.find_element(By.NAME, page_name).click()
    except Exception as e:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 10
2 페이지를 가져왔습니다 i: 20
3 페이지를 가져왔습니다 i: 30
4 페이지를 가져왔습니다 i: 40
5 페이지를 가져왔습니다 i: 50
6 페이지를 가져왔습니다 i: 60
7 페이지를 가져왔습니다 i: 70
8 페이지를 가져왔습니다 i: 80
9 페이지를 가져왔습니다 i: 90
10 페이지를 가져왔습니다 i: 100
11 페이지를 가져왔습니다 i: 110
12 페이지를 가져왔습니다 i: 120
13 페이지를 가져왔습니다 i: 130
14 페이지를 가져왔습니다 i: 140
15 페이지를 가져왔습니다 i: 150
16 페이지를 가져왔습니다 i: 160
17 페이지를 가져왔습니다 i: 170
18 페이지를 가져왔습니다 i: 180
19 페이지를 가져왔습니다 i: 190
20 페이지를 가져왔습니다 i: 200
21 페이지를 가져왔습니다 i: 210
22 페이지를 가져왔습니다 i: 220
23 페이지를 가져왔습니다 i: 230
24 페이지를 가져왔습니다 i: 240
25 페이지를 가져왔습니다 i: 250
26 페이지를 가져왔습니다 i: 260
27 페이지를 가져왔습니다 i: 270
28 페이지를 가져왔습니다 i: 280
29 페이지를 가져왔습니다 i: 290
30 페이지를 가져왔습니다 i: 300
31 페이지를 가져왔습니다 i: 310
32 페이지를 가져왔습니다 i: 320
33 페이지를 가져왔습니다 i: 330
34 페이지를 가져왔습니다 i: 340
35 페이지를 가져왔습니다 i: 350
36 페이지를 가져왔습니다 i: 360
37 페이지를 가져왔습니다 i: 370
38 페이지를 가져왔습니다 i: 380
39 페이지를 가져왔습니다 i: 390
40 페이지를 가져왔습니다 i: 400
41 페이지를 가져왔습니다 i: 410
42 페이지를 가져왔습니다 i: 420
43 페이지를 가져왔습니다 i: 430
44 페이지를 가져왔습니다 i: 440
45 페이지를 가져왔습니다 i: 450
46 페이지를 가져왔습니다 i: 460


IndexError: iloc cannot enlarge its target object

In [226]:
df.head()

,category,title,writer,date,text,hit,url
0,재테크정보,"글로벌투자, 쉬운투자, 안전한투자 3가지 모두 필요할때.",곽재혁,2023.07.27,최근 많은 금융 투자자가 급변하는 투자 환경과 확대된 금융시장의 변동성으로 투자의 ...,28379,https://omoney.kbstar.com/quics?page=C042016&b...
1,재테크정보,기초연금이 뭐예요? 신청해야 받을 수 있나요?,노지원,2023.07.18,오늘 주제 한눈에 보기 얼마전 고객님이 안내문 하나를 가지고 KB골든라이프센...,69613,https://omoney.kbstar.com/quics?page=C042016&b...
2,재테크정보,국민연금 당겨 받을까? 미뤄 받을까?,이주영 센터장,2023.06.29,퇴직을 앞둔 직장인들 사이에서 ‘국민연금 수령을 언제부터 할 것인가?’에 대한 의견...,71740,https://omoney.kbstar.com/quics?page=C042016&b...
3,재테크정보,경제상황이 썩 좋지 않은데 주가가 오르는 이유는 무엇일까?,오인석,2023.06.22,"☞유튜브 여의도5번출구 바로가기☜ 요즘 주변에 “경기가 어렵다, 불황이다”는 얘기...",53141,https://omoney.kbstar.com/quics?page=C042016&b...
4,재테크정보,몰라서 놓치는 건강보험 활용 방법,진은경 센터장,2023.06.21,오늘 주제 한눈에 보기 최근 방송인 김구라 씨가 본인 유튜브 채널에서 ‘병원도 안 ...,53154,https://omoney.kbstar.com/quics?page=C042016&b...


In [235]:
df.tail()

,category,title,writer,date,text,hit,url
457,재테크정보,주거래고객 100% 활용법,운영자,2001.10.19,최근 은행권에서는 우량고객 확보를 위하여 주거래고객 제도를 확대운영하고 있다.주거래...,15639,https://omoney.kbstar.com/quics?page=C042016&b...
458,재테크정보,저금리시대 목돈만들기 7계명,운영자,2001.09.20,목돈을 만들기 위해서는 반드시 지켜야할 몇 가지 원칙이 있다.우선 개개인이 목돈을 ...,14125,https://omoney.kbstar.com/quics?page=C042016&b...
459,재테크정보,저금리 시대의 재테크 방법,운영자,2001.08.27,우리는 지금 사상 초유의 저금리 시대를 맞이하고있다.얼마 전 까지만 해도 실질금리 ...,10321,https://omoney.kbstar.com/quics?page=C042016&b...
460,재테크정보,결혼자금 준비는 속도위반도 OK!,운영자,2001.08.08,들떠있던 휴가철도 지나 아직까지 무덥기는 하지만 입추도 지났으니 가을도 멀지 않은 ...,11315,https://omoney.kbstar.com/quics?page=C042016&b...
461,재테크정보,비과세채권투자신탁 찾을까? 말까?,운영자,2001.07.28,작년 7월 27일부터 판매하기 시작하여 2000년 말까지 한시적으로 판매한 비과세채...,3154,https://omoney.kbstar.com/quics?page=C042016&b...


In [236]:
df.to_csv('KB자산관리_재테크.csv', index = False, encoding = 'utf-8')

## 데이터 병합

In [4]:
df_부동산 = pd.read_csv("KB자산관리_부동산.csv", encoding = 'cp949')
df_재테크 = pd.read_csv("KB자산관리_재테크.csv", encoding = 'cp949')
df_세무 = pd.read_csv("KB자산관리_세무.csv", encoding = 'cp949')

In [5]:
df_통합 = pd.concat([df_부동산, df_재테크, df_세무])
df_통합.reset_index(drop = True, inplace = True)

In [16]:
print(df_부동산.shape)
print(df_재테크.shape)
print(df_세무.shape)
print("-"*10)
print(df_통합.shape)

(197, 7)
(462, 7)
(219, 7)
----------
(878, 7)


In [6]:
df_통합.head()

,category,title,writer,date,text,hit,url
0,부동산정보,보증금 깎아줄게요! 전월세 갱신계약 줄었지만 감액계약 비중 역대 최대,KB부동산,2023.07.26,서울 아파트 시세 하락이 안정을 찾고 있습니다.다만 강남권역 중심의 인기지역 외 하...,28106,https://omoney.kbstar.com/quics?page=C042015&b...
1,부동산정보,"아파트 시장의 반등, 부동산 회복될까?",박원갑,2023.07.19,"아파트 시장의 반등, 부동산 회복될까?그동안 꽁꽁 얼어붙었던 아파트 시장에 온기가 ...",50977,https://omoney.kbstar.com/quics?page=C042015&b...
2,부동산정보,박원갑 박사가 알려주는 부동산 뉴스와 광고를 제대로 읽는 법,박원갑,2023.07.06,☞ 유튜브 여의도5번출구 바로가기 ☜ '암 수술 성공 확률이 95%입니다.''암 ...,52743,https://omoney.kbstar.com/quics?page=C042015&b...
3,부동산정보,"서울 아파트값 평균 매매가격 11억대 진입, 부동산 시장의 현주소는?",골든라이프X 게시글,2023.06.23,오늘 주제 한눈에 보기서울 아파트 평균 매매가격이 11억대로 내려앉았습니다. 전국 ...,52788,https://omoney.kbstar.com/quics?page=C042015&b...
4,부동산정보,[청약제도 개편] 청약 가점이 낮아도 당첨 기회가 있다고요?,이창섭 부동산 전문위원,2023.06.20,Q1. 낮은 점수로 청약 당첨이 가능하다고요?청약제도 개편으로 가점이 낮은 세대의 ...,79587,https://omoney.kbstar.com/quics?page=C042015&b...


In [7]:
df_통합.tail()

,category,title,writer,date,text,hit,url
873,세무정보,3년 보유 2년 거주인가? 1년 거주인가?,박영선,2004.09.22,PB사업팀 전담 세무사 / 박영선 ...,240,https://omoney.kbstar.com/quics?page=C042014&b...
874,세무정보,재산세 조세저항의 원인과 전망,이장건,2004.09.07,PB 사업팀 전담 세무사 이장건) ? ...,174,https://omoney.kbstar.com/quics?page=C042014&b...
875,세무정보,"임대 주택, 이럴 때는 과세된다",윤영주,2004.08.06,윤영주(전담세무사) / GOLD&WISE PB 사업팀 ...,378,https://omoney.kbstar.com/quics?page=C042014&b...
876,세무정보,상속이 임박했을 때 할 일과 하지 말아야 할 일,이장건,2004.05.11,이장건 / GOLD&WISE 전담 세무사 ? Do's ...,834,https://omoney.kbstar.com/quics?page=C042014&b...
877,세무정보,2004년 이후 부동산 양도?상속?증여시 유의해야 할 사항,이장건,2004.05.06,이장건(전담세무사) / GOLD&WISE PB사업팀 2...,573,https://omoney.kbstar.com/quics?page=C042014&b...


In [20]:
# 텍스트 빈 행 제거 -> 이미지 파일만 있는 경우
df_통합 = df_통합[~df_통합.text.isna()]

In [29]:
df_통합.shape

(853, 7)

In [28]:
# </td> 제거
df_통합.text = df_통합.text.apply(lambda x: x.replace("</td>", ""))

In [66]:
# 전처리
df_통합.text = df_통합.text.apply(lambda x: x.replace("PB", "").replace("사업팀","").replace("(전담세무사)","").replace("GOLD&WISE","").replace("전담","").replace("세무사","").replace("/ ", "").replace(" ?", ""))

In [63]:
# 공백 제거
df_통합.text = df_통합.text.apply(lambda x: re.sub(r'\s+', ' ', x))

In [40]:
df_통합.reset_index(drop = True, inplace = True)

In [68]:
# 작성자 이름이 들어간 행들이 있음
df_통합.iloc[851,4]

"이장건 Do's and Don'ts 고액재산가일수록 상속에 대비해 절세 대책을 미리 세워 사후에 불필요한 세금 부담을 지지 않도록 할 필요가 있다. 상속세 절감 방안을 꼼꼼히 따져 보았다. 서울에 사는 자영업자 박모씨(35)는 요즘 상속 문제로 골머리를 앓고 있다. 지난해 11월 상속세 신고를 준비하면서 여러 가지 잘못한 점을 발견했기 때문이다. 박모씨의 부친은 2년간 병상에 있다 돌아가셨는데, 그동안 들어간 병원 수술비와 입원비 등 약 3억원을 부친 계좌에서 인출하여 지급하지 않고 모친 및 본인의 계좌에서 인출하여 지급하였다. 부모를 위하는 마음에서 한 일이었지만 병원비를 부친의 계좌에서 인출하여 지급한 경우에 비해 결과적으로 부친에 대한 상속세 부담이 상당히 늘어나게 되었다. 상담을 맡았던 조세전문가의 얘기는 이렇다. 부친의 상속세 최고 한계세율이 50퍼센트이므로 부친의 계좌에서 인출하여 지급한 경우에 비해 약 1억 3천 5백만 원의 상속세 부담이 늘었다는 것이다. 고액재산가들의 상속이 1~2년 이내로 임박했을 경우에는 매우 세심한 주의가 필요하다. 상속 임박 시에 절세 대책을 세우는 것도 필요하지만 상속세 부담이 늘어나지 않도록 주의하는 것도 중요하다. 상속이 임박했을 경우 할 일과 하지 말아야 할 일에 관해 점검해 보자. 상속이 임박했을 경우 해야 할 일 첫째, 금융재산가액이 10억원이 되지 않는 경우에는 금융재산가액을 10억원으로 맞춰둘 필요가 있다. 상속이 임박한 부친 명의의 금융재산가액이 5억원이고 임대용 부동산가액이 20억원이라고 가정하자. 이러한 경우 금융재산공제는 1억원밖에 되지 않는다. 그러나 임대용 부동산의 임대보증금을 5억원 상향 조정하여 이 돈을 부친 명의로 예금하면 금융재산공제가 1억원 더 늘어나게 돼 상속세를 절감할 수 있다. 단, 금융기관에서 부동산담보대출로 5억원을 빌려 부친 명의로 예금하는 경우에는 금융재산공제가 늘어나지 않는다는 점에 유의해야 한다. 금융재산공제는 금융재산가액에서 금융부채를 차감한 순금융재산가액에 대해서만 

In [70]:
# 작성자 이름 제거
df_통합.text = df_통합.apply(lambda x: x['text'].replace(x['writer'],""), axis = 1)

In [71]:
df_통합

,category,title,writer,date,text,hit,url
0,부동산정보,보증금 깎아줄게요! 전월세 갱신계약 줄었지만 감액계약 비중 역대 최대,KB부동산,2023.07.26,서울 아파트 시세 하락이 안정을 찾고 있습니다.다만 강남권역 중심의 인기지역 외 하...,28106,https://omoney.kbstar.com/quics?page=C042015&b...
1,부동산정보,"아파트 시장의 반등, 부동산 회복될까?",박원갑,2023.07.19,"아파트 시장의 반등, 부동산 회복될까?그동안 꽁꽁 얼어붙었던 아파트 시장에 온기가 ...",50977,https://omoney.kbstar.com/quics?page=C042015&b...
2,부동산정보,박원갑 박사가 알려주는 부동산 뉴스와 광고를 제대로 읽는 법,박원갑,2023.07.06,☞ 유튜브 여의도5번출구 바로가기 ☜ '암 수술 성공 확률이 95%입니다.''암 수...,52743,https://omoney.kbstar.com/quics?page=C042015&b...
3,부동산정보,"서울 아파트값 평균 매매가격 11억대 진입, 부동산 시장의 현주소는?",골든라이프X 게시글,2023.06.23,오늘 주제 한눈에 보기서울 아파트 평균 매매가격이 11억대로 내려앉았습니다. 전국 ...,52788,https://omoney.kbstar.com/quics?page=C042015&b...
4,부동산정보,[청약제도 개편] 청약 가점이 낮아도 당첨 기회가 있다고요?,이창섭 부동산 전문위원,2023.06.20,Q1. 낮은 점수로 청약 당첨이 가능하다고요?청약제도 개편으로 가점이 낮은 세대의 ...,79587,https://omoney.kbstar.com/quics?page=C042015&b...
...,...,...,...,...,...,...,...
848,세무정보,3년 보유 2년 거주인가? 1년 거주인가?,박영선,2004.09.22,"사례 서초동에 S아파트(8년 8개월 보유, 1년 8개월 거주)를 갖고 있던 A씨...",240,https://omoney.kbstar.com/quics?page=C042014&b...
849,세무정보,재산세 조세저항의 원인과 전망,이장건,2004.09.07,) 수도권을 중심으로 불거지고 있는 재산세 인상에 반발한 자치단체와 주민들의 조세...,174,https://omoney.kbstar.com/quics?page=C042014&b...
850,세무정보,"임대 주택, 이럴 때는 과세된다",윤영주,2004.08.06,작년 하반기부터 정부는 주택가격안정을 위하여 다양한 정책을 발표하였고 동 정책의 ...,378,https://omoney.kbstar.com/quics?page=C042014&b...
851,세무정보,상속이 임박했을 때 할 일과 하지 말아야 할 일,이장건,2004.05.11,Do's and Don'ts 고액재산가일수록 상속에 대비해 절세 대책을 미리 세워...,834,https://omoney.kbstar.com/quics?page=C042014&b...


In [72]:
df_통합.to_csv('KB자산관리_통합.csv', index = False, encoding = 'utf-8')